# Dependence of Fluence Predictions on the Number of Photons Simulated
**Carole Hayakawa**

**August 2025**

Goal: This exercise explores how fluence estimates change with the number of photons simulated.
It is assumed that

* [.NET 8](https://dotnet.microsoft.com/en-us/download/dotnet/8.0) has been installed

* The latest [VTS libraries](https://github.com/VirtualPhotonics/Vts.Scripting.Python/releases) have been downloaded from the zip file in releases and extracted to the libraries folder

In [ ]:
#Import the Operating System so we can access the files for the VTS library
import os
import sys
current_directory = os.getcwd()
library_directory = current_directory.replace("monte-carlo", "libraries")
vts_path = os.path.join(library_directory, "Vts.dll")
#Import Math
import math

Use pip to install PythonNet Plotly and Numpy

In [ ]:
pip install pythonnet plotly numpy ipympl

Import the Core CLR runtime from PythonNet and add the reference for the VTS library and its dependencies

Import the namespaces from the Python libraries and the VTS library

In [ ]:
from pythonnet import load
load("coreclr")
import clr
clr.AddReference(vts_path)
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# use matplotlib.pyplot
import matplotlib as mpl
import matplotlib.pyplot as plt
from Vts import *
from Vts.Common import *
from Vts.Extensions import *
from Vts.Modeling.Optimizers import *
from Vts.Modeling.ForwardSolvers import *
from Vts.SpectralMapping import *
from Vts.Factories import *
from Vts.MonteCarlo import *
from Vts.MonteCarlo.Sources import *
from Vts.MonteCarlo.Tissues import *
from Vts.MonteCarlo.Detectors import *
from Vts.MonteCarlo.Factories import *
from Vts.MonteCarlo.PhotonData import *
from Vts.MonteCarlo.PostProcessing import *
from System import Array, Double, Math
#from graph_tools import heatmap

In [ ]:
Setup the values for the simulations and plot the results using Plotly

Fluence estimates

In [ ]:
# Setup the detector input for the simulation
rhoStart = 0
rhoStop = 10  # [mm]
rhoCount = 101
zStart = 0
zStop = 10  # [mm]
zCount = 101
detectorRhoRange = DoubleRange(start=rhoStart, stop=rhoStop, number=rhoCount)
detectorZRange = DoubleRange(start=zStart, stop=zStop, number=zCount)
detectorInput = FluenceOfRhoAndZDetectorInput()
detectorInput.Rho = detectorRhoRange
detectorInput.Z = detectorZRange
detectorInput.Name = "FluenceOfRhoAndZ"
detectorInput.TallySecondMoment = True
detectors = Array.CreateInstance(IDetectorInput,1)
detectors[0] = detectorInput

# Setup the tissue input for the simulation
regions = Array.CreateInstance(ITissueRegion, 3)
regions[0] = LayerTissueRegion(zRange=DoubleRange(Double.NegativeInfinity, 0.0), op=OpticalProperties(mua=0.0, musp=1E-10, g=1.0, n=1.0)) # air
regions[1] = LayerTissueRegion(zRange=DoubleRange(0.0, 100.0), op=OpticalProperties(mua=0.01, musp=1.0, g=0.8, n=1.4)) # tissue
regions[2] = LayerTissueRegion(zRange=DoubleRange(100.0, Double.PositiveInfinity), op=OpticalProperties(mua=0.0, musp=1E-10, g=1.0, n=1.0)) # air

# Setup source
sourceInput = DirectionalPointSourceInput()
sourceInput.InitialTissueRegionIndex=0

# Setup number of photons simulated
nPhot = [10, 100, 1000, 10000]
FluenceArray = np.zeros((len(nPhot), (zCount - 1) * (rhoCount - 1)))
RelativeErrorArray = np.zeros((len(nPhot), (zCount - 1) * (rhoCount-1)))

# plot 4 cases in grid
fig, axes = plt.subplots(nrows=2,ncols=2)
xLabel = "ρ [mm]"
yLabel = "z [mm]"
title = "log(Φ(ρ,z)) [mm-2]"
# ignore divide by zero warning when calculating relative error
np.seterr(divide='ignore', invalid='ignore')

for i in range(0, len(nPhot)):
  simulationOptions = SimulationOptions()
  simulationOptions.AbsorptionWeightingType = AbsorptionWeightingType.Analog  # variation: set to Discrete
  # create a SimulationInput object to define the simulation
  simulationInput = SimulationInput()
  simulationInput.N = nPhot[i]
  simulationInput.OutputName = "MonteCarloFluence"
  simulationInput.DetectorInputs = detectors
  simulationInput.Options = simulationOptions
  simulationInput.Tissue = MultiLayerTissueInput(regions)
  # create the simulations
  simulation = MonteCarloSimulation(simulationInput)
  # run the simulations 
  simulationOutput = simulation.Run()
  # determine standard deviation and plot the results using Plotly
  detectorResults = Array.CreateInstance(FluenceOfRhoAndZDetector, 1)
  detectorResults[0] = simulationOutput.ResultsDictionary["FluenceOfRhoAndZ"]
  Fluence = Array.CreateInstance(FluenceOfRhoAndZDetector, 1)
  RelativeError = Array.CreateInstance(FluenceOfRhoAndZDetector, 1)
  FluenceArray[i] = [f for f in detectorResults[0].Mean]
  SecondMoment = [s for s in detectorResults[0].SecondMoment]
  StandardDeviation = np.sqrt((SecondMoment - np.multiply(FluenceArray[i], FluenceArray[i]) / simulationInput.N))
  # divide then set NaN to 0
  RelativeErrorArray[i] = np.divide(StandardDeviation, FluenceArray[i])
  #RelativeErrorArray[i] = np.nan_to_num(temp, nan=0)

  # plot fluence as a function of N, number of photons simulated
  # plot log of fluence and mirror fluence(rho,z) about rho=0 axis
  logFluence = [Math.Log(f) for f in FluenceArray[i]]
  # Convert to .NET array
  rhoDelta = detectorRhoRange.GetDelta()
  rhos = rhoStart + rhoDelta * np.arange(rhoCount - 1)  
  # reverse and concatenate
  allRhos = np.concatenate((-rhos[::-1], rhos))
  zDelta = detectorZRange.GetDelta()
  zs = zStart + zDelta * np.arange(zCount - 1) 
  fluenceRowsToPlot = np.array([logFluence[i:i+len(zs)] for i in range(0, len(logFluence), len(zs))])

  colormap=mpl.colormaps['magma']
  cbarTicks = [-6, -4, -2, 0]
    
  if (i==0):
    im0=allFluenceRowsToPlot = np.concatenate((fluenceRowsToPlot[::-1], fluenceRowsToPlot))
    im0=axes[0,0].imshow(allFluenceRowsToPlot.T, vmin=-6, vmax=0)
    axes[0,0].set_title('log(Flu(ρ,z))[mm^-2]');
    axes[0,0].set_xlabel('ρ [mm]')
    axes[0,0].set_ylabel('z [mm]')
    axes[0,0].text(10, 90, 'N=10')
    cbar = fig.colorbar(im0, cmap=colormap, location='right', shrink=0.6, pad=0.05)
    cbar.set_ticks(cbarTicks)
  if (i==1):
    im1=allFluenceRowsToPlot = np.concatenate((fluenceRowsToPlot[::-1], fluenceRowsToPlot))
    im1=axes[0,1].imshow(allFluenceRowsToPlot.T, vmin=-6, vmax=0)
    axes[0,1].set_title('log(Flu(ρ,z))[mm^-2]');
    axes[0,1].set_xlabel('ρ [mm]')
    axes[0,1].set_ylabel('z [mm]')
    axes[0,1].text(10, 90, 'N=100')
    cbar = fig.colorbar(im1, cmap=colormap, location='right', shrink=0.6, pad=0.05)
    cbar.set_ticks(cbarTicks)
  if (i==2):
    im2=allFluenceRowsToPlot = np.concatenate((fluenceRowsToPlot[::-1], fluenceRowsToPlot))
    im2=axes[1,0].imshow(allFluenceRowsToPlot.T, vmin=-6, vmax=0)
    axes[1,0].set_title('log(Flu(ρ,z))[mm^-2]');
    axes[1,0].set_xlabel('ρ [mm]')
    axes[1,0].set_ylabel('z [mm]')
    axes[1,0].text(10, 90, 'N=1000')
    cbar = fig.colorbar(im2, cmap=colormap, location='right', shrink=0.6, pad=0.05)
    cbar.set_ticks(cbarTicks)
  if (i==3):
    im3=allFluenceRowsToPlot = np.concatenate((fluenceRowsToPlot[::-1], fluenceRowsToPlot))
    im3=axes[1,1].imshow(allFluenceRowsToPlot.T, vmin=-6, vmax=0)
    axes[1,1].set_title('log(Flu(ρ,z))[mm^-2]');
    axes[1,1].set_xlabel('ρ [mm]')
    axes[1,1].set_ylabel('z [mm]')
    axes[1,1].text(10, 90, 'N=10000')
    cbar = fig.colorbar(im3, cmap=colormap, location='right', shrink=0.6, pad=0.05)
    cbar.set_ticks(cbarTicks)


In [ ]:
# plot relative error as a function of N, the number of photons simulated
# plot 4 cases in grid
figRelErr, axesRelErr = plt.subplots(nrows=2,ncols=2)
xLabel = "ρ [mm]"
yLabel = "z [mm]"
title = "relerror(Φ(ρ,z))"

for i in range(0, len(nPhot)):
  # plot relative error of fluence and mirror about rho=0 axis
  relativeError = [r for r in RelativeErrorArray[i]]
  relativeErrorRowsToPlot = np.array([relativeError[i:i+len(zs)] for i in range(0, len(relativeError), len(zs))])

  colormap=mpl.colormaps['magma']
  cbarTicksRelErr = [0, 0.5, 1]
    
  if (i==0):
    imRelErr0=allRelativeErrorRowsToPlot = np.concatenate((relativeErrorRowsToPlot[::-1], relativeErrorRowsToPlot))
    imRelErr0=axesRelErr[0,0].imshow(allRelativeErrorRowsToPlot.T, vmin=0, vmax=1)
    axesRelErr[0,0].set_title('relerr(Flu(ρ,z))');
    axesRelErr[0,0].set_xlabel('ρ [mm]')
    axesRelErr[0,0].set_ylabel('z [mm]')
    axesRelErr[0,0].text(10, 90, 'N=10')
    cbarRelErr = figRelErr.colorbar(imRelErr0, cmap=colormap, location='right', shrink=0.6, pad=0.05)
    cbarRelErr.set_ticks(cbarTicksRelErr)
  if (i==1):
    imRelErr1=allRelativeErrorRowsToPlot = np.concatenate((relativeErrorRowsToPlot[::-1], relativeErrorRowsToPlot))
    imRelErr1=axesRelErr[0,1].imshow(allRelativeErrorRowsToPlot.T, vmin=0, vmax=1)
    axesRelErr[0,1].set_title('relerr(Flu(ρ,z))');
    axesRelErr[0,1].set_xlabel('ρ [mm]')
    axesRelErr[0,1].set_ylabel('z [mm]')
    axesRelErr[0,1].text(10, 90, 'N=100')
    cbarRelErr = figRelErr.colorbar(imRelErr1, cmap=colormap, location='right', shrink=0.6, pad=0.05)
    cbarRelErr.set_ticks(cbarTicksRelErr)
  if (i==2):
    imRelErr2=allRelativeErrorRowsToPlot = np.concatenate((relativeErrorRowsToPlot[::-1], relativeErrorRowsToPlot))
    imRelErr2=axesRelErr[1,0].imshow(allRelativeErrorRowsToPlot.T, vmin=0, vmax=1)
    axesRelErr[1,0].set_title('relerr(Flu(ρ,z))');
    axesRelErr[1,0].set_xlabel('ρ [mm]')
    axesRelErr[1,0].set_ylabel('z [mm]')
    axesRelErr[1,0].text(10, 90, 'N=1000')
    cbarRelErr = figRelErr.colorbar(imRelErr2, cmap=colormap, location='right', shrink=0.6, pad=0.05)
    cbarRelErr.set_ticks(cbarTicksRelErr)
  if (i==3):
    imRelErr3=allRelativeErrorRowsToPlot = np.concatenate((relativeErrorRowsToPlot[::-1], relativeErrorRowsToPlot))
    imRelErr3=axesRelErr[1,1].imshow(allRelativeErrorRowsToPlot.T, vmin=0, vmax=1)
    axesRelErr[1,1].set_title('relerr(Flu(ρ,z))');
    axesRelErr[1,1].set_xlabel('ρ [mm]')
    axesRelErr[1,1].set_ylabel('z [mm]')
    axesRelErr[1,1].text(10, 90, 'N=10000')
    cbarRelErr = figRelErr.colorbar(imRelErr3, cmap=colormap, location='right', shrink=0.6, pad=0.05)
    cbarRelErr.set_ticks(cbarTicksRelErr)
    